In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setMaster("yarn").setAppName("Regression")

In [4]:
sc = SparkContext.getOrCreate (conf = conf)
sc.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'master'),
 ('spark.app.startTime', '1672130857733'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES

In [5]:
import seaborn as sns
import re
import csv
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib  import cm
import numpy as np

In [6]:
createType = "csv_read_all"
database = "train"
num_of_partition = 5

## 1 Read data from csv

In [7]:
def read_input(dataset_name, p):
    rdd = None
    if createType == "csv_read_all":
        data = []
        with open('{}.csv'.format(dataset_name), 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                data.append(tuple(row))
        cls = data[0]
        data = data[1:]
        rdd = sc.parallelize(data, num_of_partition)
        rdd = rdd.filter(lambda x: x[0] != 'Id')

    elif createType == "csv_read_chunk":
        rdd = sc.textFile("hdfs://master:9000/data/{}.csv".format(database), num_of_partition)
        rdd = rdd.mapPartitions(lambda x: csv.reader(x))
        cls = rdd.first()
        rdd = rdd.filter(lambda x: x[0] != 'Id')
    else:
        print("Can not read data")
    
    return rdd, cls

In [8]:
wholeRDD, columns = read_input(database, num_of_partition)

In [9]:
print(columns)

('Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

In [10]:
c = { attr : idx  for idx, attr in enumerate(columns) }
print("Number of columns = ", len(c))

Number of columns =  81


In [11]:
continous_cols = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 
                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
                  'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
                 'FullBath', 'HalfBath', 'TotRmsAbvGrd',
                  'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
                  'WoodDeckSF','OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
                  'PoolArea', 'MiscVal', 'MoSold', 'YrSold'
                 ]

In [12]:
category_cols = [ i for i in columns if i not in continous_cols ]
print(category_cols)

['Id', 'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'SalePrice']


In [13]:
print("Data size = ", wholeRDD.count())

Data size =  1259


In [14]:
'''
Check lines that have empty values. It seems that there are no empty values in data.
'''
print("Number of lines have empty = ", wholeRDD.filter(lambda line: not all(line)).count())

Number of lines have empty =  0


## 2.1 Process: replace NA

In [15]:
def replaceNA_normalize(line):
    new_line = list(line)
    for col in continous_cols:
        if new_line[c[col]] == 'NA':
            new_line[c[col]] = '-1'
        
        new_line[c[col]] = int(new_line[c[col]])
            
    return tuple(new_line)

In [16]:
'''
Replace all 'NA' values to '-1'. It's needed because some continuous columns might contain 'NA', which causes difficult
for calculation.
'''
wholeRDD = wholeRDD.map(lambda line: replaceNA_normalize(line))

In [17]:
def find_median(listt):
    length = len(listt)
    
    if length % 2 == 0 :
        return float(1/2 * (listt[int(length/2-1)] + listt[int(length/2-1) + 1]))
    return float(listt[int((length-1)/2)])

In [18]:
corpus_replace = {}

for col in continous_cols:
    list_values = wholeRDD.filter(lambda line: line[c[col]] != -1).map(lambda line: line[c[col]]).collect()
    median_col = find_median(sorted(list_values))
    print(col)
    print("Median = ", median_col)
    corpus_replace[col] = median_col

LotFrontage
Median =  70.0
LotArea
Median =  9452.0
YearBuilt
Median =  1973.0
YearRemodAdd
Median =  1993.0


MasVnrArea
Median =  0.0
BsmtFinSF1
Median =  385.0
BsmtFinSF2
Median =  0.0


BsmtUnfSF
Median =  470.0
TotalBsmtSF
Median =  996.0
1stFlrSF
Median =  1086.0
2ndFlrSF
Median =  0.0


LowQualFinSF
Median =  0.0
GrLivArea
Median =  1456.0
BsmtFullBath
Median =  0.0
BsmtHalfBath
Median =  0.0
FullBath
Median =  2.0
HalfBath
Median =  0.0
TotRmsAbvGrd
Median =  6.0
Fireplaces
Median =  1.0
GarageYrBlt
Median =  1980.0
GarageCars
Median =  2.0
GarageArea
Median =  477.0
WoodDeckSF
Median =  0.0
OpenPorchSF
Median =  24.0
EnclosedPorch
Median =  0.0
3SsnPorch
Median =  0.0
ScreenPorch
Median =  0.0
PoolArea
Median =  0.0
MiscVal
Median =  0.0
MoSold
Median =  6.0
YrSold
Median =  2008.0


## 2.2 Process: Label encode

In [19]:
def label_encode(line):
    new_line = list(line)
    for col in continous_cols:
        if new_line[c[col]] != -1:     # -1 is NA, I want to keep this values instead 
            
            if new_line[c[col]] > corpus_replace[col]:
                new_line[c[col]] = 2 # High

            else:
                new_line[c[col]] = 1 # Low

    return tuple(new_line)

In [20]:
wholeRDD = wholeRDD.map(lambda line: label_encode(line))

## 3. Build model

In [21]:
'''
The last columns (predictions also need to be converted to numerics)
'''
def process_sale_price(line):
    new_line = list(line)
    new_line[c['SalePrice']] = float(new_line[c['SalePrice']])
    return tuple(new_line)

wholeRDD = wholeRDD.map(lambda line: process_sale_price(line))

In [22]:
wholeRDD.map(lambda x: x[c['SalePrice']]).reduce(lambda a,b : a + b)

228061319.0

In [23]:
def find_count_sum_mean_std(rdd, conditions, column):
        
    # For all 
    if column == None:
        sum_count = rdd
        if conditions != None:
            sum_count = sum_count.filter(conditions)  
            
        sum_count = sum_count.map(lambda x: (x[c['SalePrice']], 1)).reduce(lambda a,b: (a[0] + b[0], a[1] + b[1]))
        sum_all = sum_count[0]
        count = sum_count[1]
        mean = sum_all / count
        
        variance = rdd
        if conditions != None:
            variance = variance.filter(conditions)
            
        variance = variance.map(lambda x: (x[c['SalePrice']] - mean) * (x[c['SalePrice']] - mean) / count).reduce(lambda a,b : a + b)
     
        std = math.sqrt(variance)

        return sum_all, count, mean, std

    else:
        sum_count = rdd
        # For each column, column must be not None
        if conditions != None:
            sum_count = sum_count.filter(conditions)
            
        sum_count = sum_count.map(lambda x: ( x[c[column]],(x[c['SalePrice']], 1))).reduceByKey(lambda a,b: (a[0] + b[0], a[1] + b[1]))
        # (attribute_name, (sum_of_attribute, count_of_attribute))
        
        sum_count_mean = sum_count.map(lambda x: (x[0], (x[1][0], x[1][1], x[1][0] / x[1][1])))
        #(name, (sum, count, mean))
        
        
        variance = rdd
        if conditions != None:
            variance = variance.filter(conditions)
            
        variance = variance.map(lambda line: (line[c[column]], line[c['SalePrice']])).join(sum_count_mean)
        # (attribute_name, (price, (sum, count, mean)))
        
        variance = variance.map(lambda x: (x[0], (x[1][0]-x[1][1][2])*(x[1][0]-x[1][1][2])/x[1][1][1])).reduceByKey(lambda a,b : a+b )
        # (attribute_name, variance)
        
        result = variance.join(sum_count_mean).map(lambda x: (x[0], x[1][1][0], x[1][1][1], x[1][1][2], math.sqrt(x[1][0]))).collect()
        
        return result
        

In [24]:
# test_rdd = sc.parallelize([(1, 'Sunny', 'Hot', 'High', 'Weak', 25),
#                            (2, 'Sunny', 'Hot', 'High', 'Strong', 30),
#                            (3, 'Overcast', 'Hot', 'High', 'Weak', 46),
#                            (4, 'Rain', 'Mild', 'High', 'Weak', 45),
#                            (5, 'Rain', 'Cool', 'Normal', 'Weak', 52),
#                            (6, 'Rain', 'Cool', 'Normal', 'Strong', 23),
#                            (7, 'Overcast', 'Cool', 'Normal', 'Strong', 43),
#                            (8, 'Sunny','Mild','High','Weak',35),
#                            (9, 'Sunny','Cool','Normal','Weak',38),
#                            (10, 'Rain','Mild','Normal','Weak',46),
#                            (11, 'Sunny','Mild','Normal','Strong',48),
#                            (12, 'Overcast','Mild','High','Strong',52),
#                            (13, 'Overcast','Hot','Normal','Weak',44),
#                            (14, 'Rain','Mild','High','Strong',30),
#                           ])
# c = {'Id': 0, 'Outlook': 1, 'Temp': 2, 'Humidity': 3, 'Wind': 4, 'SalePrice': 5}
# columns = ['Id', 'Outlook', 'Temp', 'Humidity', 'Wind', 'SalePrice']
# excluded_cols = ['Id','SalePrice']

In [25]:
excluded_cols = ['Id', 'SalePrice']
valid_node_to_expand = [i for i in columns if i not in excluded_cols]

In [26]:
class Node:
    def __init__(self, col_name, col_infor, _node_to_expand = None, parent = None, parent_attribute = None):
        self.name = col_name
        self.infor = col_infor
        self.parent = parent
        self.parent_attribute = parent_attribute
        
        self.node_to_expand = []
        if _node_to_expand != None:
            self.node_to_expand = _node_to_expand
        
        
    def printt(self):
        print("name node = ", self.name , "\n")
        print("node infor \n")
    
        for v in self.infor:
            v.printt()
        
        print("expand nodes = ", self.node_to_expand)
        print("parent = ", self.parent)
        print("parent attribute = ", self.parent_attribute)
        
class Attribute:
    def __init__(self, node_name, attr):
        self.node = node_name
        self.name = attr[0]
        self.sum = attr[1]
        self.count = attr[2]
        self.mean = attr[3]
        self.std = attr[4]
    
    def printt(self):
        
        print("attr node = ",self.node)
        print("attr name = ",self.name)
        print("attr sum = ", self.sum)
        print("attr count = ",self.count)
        print("attr mean = ", self.mean)
        print("attr std = ", self.std)
        print("\n")

In [27]:
len(valid_node_to_expand)

79

In [28]:
print(find_count_sum_mean_std(wholeRDD, None, None))

(228061319.0, 1259, 181144.81254964258, 80556.01333981183)


In [29]:
'''
For short, I only use 3 columns + column ID and column SalePrice to make regression
'''
valid_node_to_expand = ['MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape']

In [30]:
'''
Data now is ready for model. 
'''
checkpoint = {}
max_reduced_std = -math.inf
candidate_root = None
# Find root node first

sum_total, count_total, mean_total, std_total = find_count_sum_mean_std(wholeRDD, None, None)

for col in valid_node_to_expand:
    if col not in excluded_cols:
        # [(name, summ_col, count_col, mean_col, std_col)]
        result = find_count_sum_mean_std(wholeRDD, None, col)
        
        weighted_std = 0
        for value in result:
            weighted_std = weighted_std + value[2] / count_total * value[4]
        reduced_std = std_total - weighted_std
        if reduced_std > max_reduced_std:
            max_reduced_std = reduced_std
            result = [Attribute(col, value) for value in result]
            
            _expand_node = valid_node_to_expand.copy()
            _expand_node.remove(col)
            
            candidate_root = Node(col, result, _expand_node, None, None)
            
# after run through all columns, find root by selecting max reduced_std, save information of root node to dictionary
checkpoint['layer_{}'.format(0)] = [candidate_root]

In [31]:
for node in checkpoint['layer_0']:
    node.printt()

name node =  LotArea 

node infor 

attr node =  LotArea
attr name =  1
attr sum =  95333195.0
attr count =  630
attr mean =  151322.53174603175
attr std =  52916.51059289869


attr node =  LotArea
attr name =  2
attr sum =  132728124.0
attr count =  629
attr mean =  211014.50556438792
attr std =  91658.43981920595


expand nodes =  ['MSZoning', 'LotFrontage', 'Street', 'LotShape']
parent =  None
parent attribute =  None


In [32]:
valid_node_to_expand

['MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape']

In [33]:
def find_weighted_std(result, count_total):
    weighted_std = 0
    
    for value in result:
        weighted_std = weighted_std + value[2] / count_total * value[4]
    
    return weighted_std

In [34]:
current_to_expand = 0
stop_condition = 10

while True:
    
    print("Constructing for layer {}".format(current_to_expand + 1))
    
    checkpoint['layer_{}'.format(current_to_expand + 1)] = [] # List of nodes in constructing layer
    # Iterate through each nodes in layers
    for idx, node in enumerate(checkpoint['layer_{}'.format(current_to_expand)]):
        node_name = node.name
        node_attributes = node.infor
        node_to_expandd = node.node_to_expand
        
        # Iterate through each attributes of a node
        for attr in node_attributes:
            
            col_name = attr.node
            
            assert col_name == node_name, "Attributes must match with parent"
            
            _name, sum_total, count_total, mean_total, std_total = attr.name, attr.sum, attr.count, attr.mean, attr.std
            
            candidate_node = None
            max_reduced_std = -math.inf
            
            if attr.count > stop_condition:
                for _node in node_to_expandd:
                    result = find_count_sum_mean_std(wholeRDD, lambda line: line[c[col_name]] == _name, _node)
                          
                    reduced_std = std_total - find_weighted_std(result, count_total)
                    
                    if reduced_std > max_reduced_std:
                        max_reduced_std = reduced_std
                        result = [Attribute(_node, value) for value in result]

                        _expand_node = node_to_expandd.copy()
                        _expand_node.remove(_node)

                        candidate_node = Node(_node, result, _expand_node, node_name + "_" + str(idx), _name)
                
                if candidate_node != None:
                    checkpoint['layer_{}'.format(current_to_expand + 1)].append(candidate_node)
            else:
                print("Stop expanding tree")
                print(mean_total)
    
#     for v in checkpoint['layer_{}'.format(current_to_expand + 1)]:
#         v.printt()
#         print("\n")
    
    if len(checkpoint['layer_{}'.format(current_to_expand + 1)]) == 0:
        break
        
    current_to_expand = current_to_expand + 1
    
    print("\n")
    print("\n")

Constructing for layer 1






Constructing for layer 2


Stop expanding tree
69697.14285714286






Constructing for layer 3


Stop expanding tree
292500.0
Stop expanding tree
73000.0


...

Stop expanding tree
73985.0


Stop expanding tree
85000.0
Stop expanding tree
142500.0
Stop expanding tree
232063.5


Stop expanding tree
267845.0


Stop expanding tree
292500.0
Stop expanding tree
73000.0






Constructing for layer 5
Stop expanding tree
110000.0
Stop expanding tree
110548.25
Stop expanding tree
228950.0
Stop expanding tree
110548.25
Stop expanding tree
228950.0
Stop expanding tree
110000.0
Stop expanding tree
110548.25
Stop expanding tree
228950.0
Stop expanding tree
121346.5
Stop expanding tree
103166.66666666667
Stop expanding tree
228950.0
Stop expanding tree
121346.5
Stop expanding tree
103166.66666666667
Stop expanding tree
228950.0
Stop expanding tree
121346.5
Stop expanding tree
103166.66666666667
Stop expanding tree
228950.0
Stop expanding tree
110000.0
Stop expanding tree
110548.25
Stop expanding tree
228950.0
Stop expanding tree
110548.25
Stop expanding tree
228950.0
Stop expanding tree
178050.0
Stop expanding tree
178050.0
Stop expanding tree
110000.0
Stop expanding tree
178050.0
Stop expanding tree
110000.0
Stop expanding tree
178050.0
Stop expanding tree
110000.0
Stop expanding tree
178050.0
Stop expanding tree
110000.0
Stop expanding tree
110000.0
Stop exp

In [35]:
for info in checkpoint['layer_0'][0].infor:
    info.printt()

attr node =  LotArea
attr name =  1
attr sum =  95333195.0
attr count =  630
attr mean =  151322.53174603175
attr std =  52916.51059289869


attr node =  LotArea
attr name =  2
attr sum =  132728124.0
attr count =  629
attr mean =  211014.50556438792
attr std =  91658.43981920595




In [36]:
def make_condition(line, path):
    cond = True
    for i in range(len(path)):
        if i % 2 == 0:
            cond = cond and line[c[path[i].name]] == path[i+1].name
    return cond

In [37]:
def predict(model, item):
    current_layer = 0
    node_to_expand = None
    index_of_parent = None
    attribute_to_expand = None
    path = []
    condition = None
    
    while True:
        layer_name = 'layer_{}'.format(current_layer)
        
        node_list = model[layer_name]
        
        
        if current_layer == 0 :
            assert len(node_list) == 1, "Root is only one"
            
            node_to_expand = node_list[0]
            index_of_parent = 0
            
            assert node_to_expand.parent_attribute == None , "Not root !!!!!!!!!!"
            assert node_to_expand.parent == None, "Not root !!!!!!!!"

        else:
            
            assert node_to_expand != None, "Parent node must be not None !!!!!!!!!"
            assert attribute_to_expand != None, "Attribute node must be not None !!!!!!!!"
            name = node_to_expand.name
            node_to_expand = None
            for idx, node in enumerate(node_list):
                # Check match parent node and respective attribute
                if node.parent == name + "_" + str(index_of_parent) and attribute_to_expand.name == node.parent_attribute:
                    
                    # Assign new node to expand
                    node_to_expand = node    
                    index_of_parent = idx
                    break
        
        if node_to_expand == None:
            break
            
        # Find attribute to expand
        name = node_to_expand.name
        attribute_list = node_to_expand.infor   
        check_value = item[c[name]]
        attribute_to_expand = None
        
        for attr in attribute_list:
            if attr.name == check_value:
                attribute_to_expand = attr
                break          
        
        path.append(node_to_expand)
        
        if attribute_to_expand != None:
            path.append(attribute_to_expand)
        
        current_layer = current_layer + 1
        
    # Based on path to make prediction
    return path

In [38]:
def make_prediction(rdd, path):
    
    result = rdd.filter(lambda line: make_condition(line, path)).map(lambda x: (x[c['SalePrice']], 1))
    
    try:
        result = result.reduce(lambda a,b : (a[0] + b[0], a[1] + b[1]))
    except:
        result = rdd.map(lambda x: (x[c['SalePrice']], 1)).reduce(lambda a,b : (a[0] + b[0], a[1] + b[1]))
        
    return result[0] / result[1]

In [39]:
testRDD, _ = read_input("test", 2)

In [40]:
print("Number of columns in test dataset = ", len(testRDD.take(1)[0]))

Number of columns in test dataset =  80


In [41]:
testRDD = testRDD.map(lambda line: label_encode(replaceNA_normalize(line)))
test_items = testRDD.collect()

In [42]:
for idx, test_item in enumerate(test_items):
    _path = predict(checkpoint, test_item)
    print(test_item[c['Id']], make_prediction(wholeRDD, _path))

1260 136057.125
1261 163500.0
1262 213527.23383084577
1263 208577.27272727274
1264 136057.125
1265 112642.85714285714
1266 175111.65
1267 202989.5
1268 261423.55371900825
1269 184523.4301075269
1270 213527.23383084577
1271 208577.27272727274
1272 162101.27659574468
1273 184523.4301075269
1274 261423.55371900825
1275 152311.39705882352
1276 213527.23383084577
1277 184523.4301075269
1278 184523.4301075269
1279 213527.23383084577
1280 69697.14285714286
1281 216175.17307692306
1282 170587.75925925927
1283 170587.75925925927
1284 165200.34426229508
1285 136057.125
1286 112157.20338983051
1287 208577.27272727274
1288 184523.4301075269
1289 170587.75925925927
1290 261423.55371900825
1291 184523.4301075269
1292 112157.20338983051
1293 112157.20338983051
1294 213527.23383084577
1295 152311.39705882352
1296 152311.39705882352
1297 165200.34426229508
1298 112157.20338983051
1299 302908.3333333333
1300 165200.34426229508
1301 184523.4301075269
1302 162101.27659574468
1303 213527.23383084577
1304 1